In [1]:
from pymongo import MongoClient

In [ ]:
class TwitterSearchApp:
    def __init__(self, max_cache_size=100):
        self.max_cache_size = max_cache_size
        self.cache = {}
        self.client = MongoClient('mongodb://localhost:27017/')
        self.cache_collection = self.client['twitter_db']['cache']

    def load_cache_from_mongodb(self):
        cache_data = self.cache_collection.find_one()
        if cache_data:
            self.cache = cache_data['cache']

    def search_tweets(self, keyword):
        if keyword in self.cache:
            print("Result found in cache:")
            return self.cache[keyword]
        else:
            print("Searching in MongoDB:")
            # Placeholder for MongoDB query, replace with actual query to search for tweets
            result = self.query_mongodb(keyword)
            if len(self.cache) < self.max_cache_size:
                self.cache[keyword] = result
                self.cache_collection.update_one({}, {'$set': {'cache': self.cache}}, upsert=True)
            return result

    def query_mongodb(self, keyword):
        # Placeholder for MongoDB query, replace with actual query to search for tweets
        return f"Dummy result for {keyword}"

    def shutdown(self):
        self.cache_collection.update_one({}, {'$set': {'cache': self.cache}}, upsert=True)
        self.client.close()

# Example usage
if __name__ == "__main__":
    app = TwitterSearchApp(max_cache_size=10)
    app.load_cache_from_mongodb()

    # Example searches
    print(app.search_tweets("python"))
    print(app.search_tweets("mongodb"))
    print(app.search_tweets("data science"))

    app.shutdown()